In [1]:
import pandas as pd

In [3]:
header=['user_id', 'item_id', 'rating']
content_data = pd.read_csv('metaSim.csv', sep=',', names=header)
tag_data = pd.read_csv('tagsSim.csv', sep=',', names=header)
cf_data = pd.read_csv('cf_pre.csv', sep=',', names=header)
svd_data = pd.read_csv('lfm_pre.csv', sep=',', names=header)

In [4]:
header=['user_id', 'item_id', 'rating', 'timestamp']
train_data = pd.read_csv('train.csv', sep=',', names=header)
test_data = pd.read_csv('test.csv', sep=',' , names=header)

In [6]:
r={} #movie by user
content_sim={} #content based
tag_sim={} #based on tag
cf_pre={} #collaborative filtering
svd_pre={} #latent factor

In [9]:
for line in train_data.itertuples():        
    if line[2] in r:
        r[line[2]].update({line[1]: line[3]})
    else:
        r.update({line[2]: {line[1]: line[3]}})

for line in content_data.itertuples():        
    if line[2] in content_sim:
        content_sim[line[2]].update({line[1]: line[3]})
    else:
        content_sim.update({line[2]: {line[1]: line[3]}})

for line in tag_data.itertuples():        
    if line[2] in tag_sim:
        tag_sim[line[2]].update({line[1]: line[3]})
    else:
        tag_sim.update({line[2]: {line[1]: line[3]}})

for line in cf_data.itertuples():        
    if line[2] in cf_pre:
        cf_pre[line[2]].update({line[1]: line[3]})
    else:
        cf_pre.update({line[2]: {line[1]: line[3]}})

for line in svd_data.itertuples():        
    if line[2] in svd_pre:
        svd_pre[line[2]].update({line[1]: line[3]})
    else:
        svd_pre.update({line[2]: {line[1]: line[3]}})

In [57]:
import numpy as np
import random
w=np.asarray([1, 1, 1, 1])
episilon=0.1
step=0.001
loop=10
for i in range(loop):
    error=0
    num=0
    for movie in r:
        n=len(r[movie])
        loss=0
        dw=[0, 0, 0, 0]
        for user in r[movie]:
            x=[]
            if user in content_sim and movie in content_sim[user]: x.append(content_sim[user][movie])
            else: x.append(0)            
            if user in tag_sim and movie in tag_sim[user]: x.append(tag_sim[user][movie])
            else: x.append(0)            
            if movie in cf_pre and user in cf_pre[movie]: x.append(cf_pre[movie][user])
            else: x.append(2.5)                
            if movie in svd_pre and user in svd_pre[movie]: x.append(svd_pre[movie][user])
            else: x.append(2.5)
          
            x=np.asarray(x)
            dw=dw+(np.dot(w, x)-r[movie][user])*x+w
            loss=loss+(np.dot(w, x)-r[movie][user])**2
        error=error+loss
        num=num+n
        loss=loss/n
        dw=dw/n
        w=w-step*dw
        # print('loss:', loss)
    error=error/num
    print('No.', i, 'trained successfully!', 'The main error is: ', error)
    if error<episilon:
        break

No. 0 trained successfully! The main error is:  0.6119945616854807
No. 1 trained successfully! The main error is:  0.30239998623949244
No. 2 trained successfully! The main error is:  0.30239979240509846
No. 3 trained successfully! The main error is:  0.3023997924052048
No. 4 trained successfully! The main error is:  0.30239979240520487
No. 5 trained successfully! The main error is:  0.30239979240520487
No. 6 trained successfully! The main error is:  0.30239979240520487
No. 7 trained successfully! The main error is:  0.30239979240520487
No. 8 trained successfully! The main error is:  0.30239979240520487
No. 9 trained successfully! The main error is:  0.30239979240520487


In [58]:
print(w)

[1.28231347e-04 1.87106411e-06 3.10075898e-01 6.70507815e-01]


In [ ]:
for line in train_data.itertuples():    
    user=line[1]
    movie=line[2]
    rate=line[3]
    x=[]
    if movie in content_sim and user in content_sim[movie]:
        x.append(content_sim[movie][user])
    else:
        x.append(0)
            
    if movie in tag_sim and user in tag_sim[movie]:
        x.append(tag_sim[movie][user])
    else:
        x.append(0)
                
    if movie in cf_pre and user in cf_pre[movie]:
        x.append(cf_pre[movie][user])
    else:
        x.append(2.5)
                
    if movie in svd_pre and user in svd_pre[movie]:
        x.append(svd_pre[movie][user])
    else:
        x.append(2.5)
    x=np.asarray(x)
    prediction=np.dot(w, x)
    print(user, '\t', movie, '\t', rate, '\t', prediction)

1 	 1 	 4.0 	 2.9013866216320987
1 	 3 	 4.0 	 4.048918734276321
1 	 6 	 4.0 	 4.301457980521397
1 	 47 	 5.0 	 4.781041447540014
1 	 50 	 5.0 	 3.1689493880152546
1 	 70 	 3.0 	 3.4350609975217234
1 	 101 	 5.0 	 4.828605379537617
1 	 110 	 4.0 	 3.1066668121001006
1 	 151 	 5.0 	 4.306327528004291
1 	 157 	 5.0 	 4.305978735783211
1 	 163 	 5.0 	 3.839282942828221
1 	 216 	 5.0 	 4.568602086892935
1 	 223 	 3.0 	 3.9641643436506664
1 	 231 	 5.0 	 4.291642405540201
1 	 235 	 4.0 	 4.285004450260741
1 	 260 	 5.0 	 4.854187006373769
1 	 296 	 3.0 	 4.295271466640822
1 	 316 	 3.0 	 3.812495279565141
1 	 333 	 5.0 	 4.654019274044997
1 	 349 	 4.0 	 4.174280172412769
1 	 356 	 4.0 	 4.8470053093104735
1 	 362 	 5.0 	 4.506140267340788
1 	 367 	 4.0 	 4.126233915030209
1 	 423 	 3.0 	 3.495826576021547
1 	 441 	 4.0 	 4.224648531587656
1 	 457 	 5.0 	 4.867791754906799
1 	 480 	 4.0 	 4.074927010803078
1 	 500 	 3.0 	 3.6154558105991614
1 	 527 	 5.0 	 4.744100560805052
1 	 543 	 4.0 	 

2 	 91529 	 3.5 	 4.24470799757948
2 	 91658 	 2.5 	 3.079748733360014
2 	 99114 	 3.5 	 2.6964480540361797
2 	 106782 	 5.0 	 2.6751067833214313
2 	 109487 	 3.0 	 3.520180997241469
2 	 112552 	 4.0 	 4.0486192685990225
2 	 114060 	 2.0 	 2.779796109163858
2 	 115713 	 3.5 	 3.6858872806272243
2 	 122882 	 5.0 	 4.179405870917904
2 	 131724 	 5.0 	 3.348042306076816
3 	 31 	 0.5 	 1.3788250176312984
3 	 527 	 0.5 	 0.43139144196833623
3 	 647 	 0.5 	 1.5668326561488437
3 	 688 	 0.5 	 1.86563526151047
3 	 720 	 0.5 	 1.584277669997855
3 	 849 	 5.0 	 3.613566779355413
3 	 914 	 0.5 	 1.5915199832462128
3 	 1093 	 0.5 	 1.2094239462937992
3 	 1124 	 0.5 	 1.377918655074054
3 	 1263 	 0.5 	 1.5573863114131266
3 	 1272 	 0.5 	 1.6625416095984829
3 	 1275 	 3.5 	 3.15983213621538
3 	 1302 	 0.5 	 1.312492644632484
3 	 1371 	 3.0 	 2.678249181099228
3 	 1587 	 4.5 	 3.247167450385978
3 	 2018 	 0.5 	 1.3406095593399623
3 	 2080 	 0.5 	 1.2675479535867509
3 	 2090 	 0.5 	 1.3846959815734554

6 	 291 	 3.0 	 3.129646778916707
6 	 292 	 4.0 	 2.514716785986899
6 	 293 	 3.0 	 4.021060072412089
6 	 302 	 3.0 	 3.112210619462141
6 	 303 	 3.0 	 3.0513665229299507
6 	 312 	 1.0 	 1.5226009231455875
6 	 315 	 4.0 	 3.517494943625833
6 	 316 	 5.0 	 4.092815153573196
6 	 330 	 3.0 	 3.0919351562619486
6 	 333 	 5.0 	 4.29063958128985
6 	 337 	 5.0 	 4.140921017017619
6 	 340 	 4.0 	 3.696967319087721
6 	 342 	 3.0 	 3.2601950582382893
6 	 343 	 3.0 	 3.050608608770402
6 	 344 	 3.0 	 3.607428155614508
6 	 347 	 3.0 	 3.2167233549155334
6 	 348 	 3.0 	 2.991453749075443
6 	 353 	 4.0 	 3.3570525907750373
6 	 355 	 4.0 	 3.4896433523412944
6 	 357 	 3.0 	 3.235344468123327
6 	 360 	 3.0 	 3.0471293761138467
6 	 361 	 4.0 	 3.5341767919680223
6 	 366 	 3.0 	 3.3498786061854178
6 	 370 	 4.0 	 3.5825985830871065
6 	 371 	 3.0 	 3.4748835173984602
6 	 377 	 5.0 	 4.555203808051384
6 	 382 	 3.0 	 3.239694850174856
6 	 405 	 3.0 	 2.6440887207758657
6 	 410 	 3.0 	 2.1484780923911133
6

8 	 592 	 3.0 	 3.178014384862878
8 	 593 	 4.0 	 4.4446505491086015
8 	 597 	 3.0 	 3.1847483064949857
9 	 41 	 3.0 	 3.192659241997724
9 	 187 	 3.0 	 3.3958442831439175
9 	 223 	 4.0 	 3.7170836805191683
9 	 371 	 3.0 	 2.9648018839399395
9 	 627 	 3.0 	 3.08782475671752
9 	 922 	 4.0 	 3.91971413651526
9 	 923 	 5.0 	 4.236151327883986
9 	 1037 	 2.0 	 2.3600410131136345
9 	 1095 	 4.0 	 3.63294656512548
9 	 1198 	 5.0 	 4.400730806818144
9 	 1270 	 5.0 	 4.319056986022518
9 	 1674 	 4.0 	 3.473888666831657
9 	 1987 	 2.0 	 2.5751187764463705
9 	 2011 	 3.0 	 3.1739948404098595
9 	 2012 	 4.0 	 3.5683757258796467
9 	 2023 	 3.0 	 3.108939872336382
9 	 2300 	 5.0 	 4.144364870068214
9 	 2877 	 3.0 	 2.9633541321109913
9 	 2901 	 2.0 	 2.455250699426127
9 	 3173 	 2.0 	 2.4692646136989356
9 	 3328 	 3.0 	 3.402330641996818
9 	 3735 	 3.0 	 3.2174232545916395
9 	 4131 	 1.0 	 1.9739164893519379
9 	 4558 	 1.0 	 2.000939061225618
9 	 4993 	 5.0 	 4.214403758581979
9 	 5218 	 4.0 	 3.50

16 	 2858 	 3.5 	 2.3952553441671762
16 	 2959 	 3.5 	 2.5268172368411204
16 	 3000 	 4.0 	 3.709336782626636
16 	 3022 	 3.5 	 3.515672205207167
16 	 3108 	 4.0 	 3.577975737450873
16 	 3174 	 3.5 	 3.4245147245911958
16 	 3429 	 3.5 	 3.8716986636104602
16 	 3683 	 4.0 	 3.881962437462079
16 	 3741 	 4.0 	 3.6243887507894303
16 	 3949 	 4.0 	 3.724973063662854
16 	 4002 	 3.5 	 3.5569206620239058
16 	 4011 	 3.0 	 3.5193932490285484
16 	 4226 	 4.5 	 2.52088509209564
16 	 4878 	 3.5 	 2.477927785643455
16 	 4973 	 4.0 	 3.9167793374359636
16 	 4993 	 3.5 	 2.429733850333746
16 	 5618 	 4.5 	 3.977588747097589
16 	 5782 	 4.0 	 3.837291919361115
16 	 5952 	 3.5 	 2.3624528743431252
16 	 5971 	 4.0 	 3.7611965817348327
16 	 6350 	 3.5 	 3.6965499496131375
16 	 7099 	 3.5 	 3.607350509517632
16 	 7153 	 3.5 	 2.394830725576439
16 	 7361 	 3.5 	 2.6009832764417253
16 	 27611 	 4.0 	 3.5770141644505973
16 	 27773 	 4.0 	 3.7490798429664114
16 	 48394 	 4.0 	 3.046086157637172
16 	 58559 	

...

132 	 318 	 4.5 	 2.715479602031843
132 	 319 	 2.5 	 2.838689461734173
132 	 337 	 3.0 	 3.36168753004135
132 	 344 	 1.5 	 2.735391548215506
132 	 345 	 4.0 	 3.500902681551452
132 	 353 	 3.0 	 3.004349844721725
132 	 355 	 2.0 	 1.918534189479976
132 	 356 	 4.0 	 2.4561723251570164
132 	 357 	 3.5 	 3.4006267254791913
132 	 364 	 4.5 	 3.8526529627108315
132 	 390 	 2.0 	 2.5584671885682027
132 	 410 	 3.0 	 3.0717001906702412
132 	 497 	 2.0 	 2.596990331622668
132 	 500 	 2.5 	 2.7678671321888904
132 	 520 	 1.5 	 2.094610501049014
132 	 539 	 3.0 	 2.024016425429102
132 	 551 	 3.5 	 3.368069694674555
132 	 555 	 3.5 	 3.5221863326546554
132 	 562 	 2.5 	 2.72097310812678
132 	 567 	 2.0 	 2.075352458155569
132 	 586 	 2.0 	 2.116190707700893
132 	 588 	 3.5 	 3.2907423547898467
132 	 592 	 2.0 	 2.5805649589348496
132 	 593 	 3.5 	 2.4750032296785727
132 	 594 	 3.0 	 2.7833534319253066
132 	 595 	 4.5 	 3.692916101659809
132 	 596 	 2.0 	 1.771314844510841
132 	 617 	 2.5 	 2

132 	 45880 	 3.0 	 2.966095255482625
132 	 46578 	 4.0 	 2.384852928079006
132 	 46976 	 4.0 	 3.692141866306458
132 	 48082 	 3.5 	 3.31006417587037
132 	 48394 	 3.0 	 2.251244160770738
132 	 48516 	 3.0 	 2.3011544370613013
132 	 48780 	 3.5 	 3.5037228169835135
132 	 48997 	 3.0 	 2.8900512659986908
132 	 50872 	 3.5 	 3.218403695722876
132 	 52281 	 4.0 	 3.49536115841644
132 	 55269 	 2.0 	 2.4242233881072863
132 	 56367 	 3.0 	 3.1104884963098125
132 	 56782 	 2.5 	 3.0978680072440907
132 	 58559 	 3.5 	 2.4567560107721618
132 	 60069 	 0.5 	 2.1964767187991012
132 	 63082 	 3.5 	 3.266569164295376
132 	 64839 	 2.5 	 2.8078976788466035
132 	 68157 	 3.5 	 3.4580535213535883
132 	 68954 	 3.5 	 3.730689290017624
132 	 79132 	 3.0 	 3.7486636097925046
133 	 32 	 4.0 	 3.509336676860393
133 	 110 	 3.0 	 2.238130364197672
133 	 288 	 2.0 	 3.2997449007243835
133 	 300 	 3.0 	 3.531605416749775
133 	 377 	 3.0 	 2.070028219209939
133 	 454 	 3.0 	 2.0844850568141324
133 	 471 	 4.

137 	 780 	 3.0 	 3.732652988819659
137 	 858 	 5.0 	 4.595043909842396
137 	 903 	 4.0 	 3.9369665148442476
137 	 904 	 4.0 	 4.070723386415145
137 	 912 	 5.0 	 4.44072575608088
137 	 913 	 4.5 	 4.226321612102911
137 	 922 	 5.0 	 4.4848265189916905
137 	 924 	 4.0 	 2.6468313429922494
137 	 954 	 4.5 	 4.163286342919449
137 	 955 	 3.0 	 3.526707059719522
137 	 968 	 3.5 	 3.683552546843937
137 	 969 	 4.0 	 4.042843598897179
137 	 1136 	 5.0 	 2.9526234838450804
137 	 1193 	 4.0 	 2.7847476602906007
137 	 1196 	 4.0 	 4.2553406861979735
137 	 1197 	 3.5 	 2.7098656803288352
137 	 1200 	 4.0 	 3.7171867883702845
137 	 1203 	 4.5 	 4.248866873309188
137 	 1207 	 3.0 	 3.7220633120244493
137 	 1208 	 5.0 	 4.379419639822211
137 	 1210 	 3.0 	 4.023474357409335
137 	 1219 	 5.0 	 2.9085431466507883
137 	 1221 	 5.0 	 4.448953678256371
137 	 1225 	 5.0 	 4.415764684880563
137 	 1228 	 4.5 	 4.27898673937271
137 	 1231 	 4.0 	 3.834112351113035
137 	 1233 	 4.5 	 4.2674810620488
137 	 1

140 	 50 	 3.0 	 3.8227232854362407
140 	 62 	 4.5 	 4.513832626401202
140 	 86 	 4.0 	 3.9552335454288885
140 	 95 	 2.5 	 3.1172114153495785
140 	 104 	 3.0 	 3.361288858189557
140 	 105 	 4.0 	 4.035827851254883
140 	 110 	 4.0 	 4.785385672358638
140 	 146 	 3.0 	 2.8315943071653136
140 	 150 	 5.0 	 4.861403830646747
140 	 151 	 4.0 	 2.8280884129471864
140 	 161 	 4.0 	 3.91010738918101
140 	 163 	 3.0 	 1.8445947190285514
140 	 164 	 3.0 	 3.0858448673430634
140 	 165 	 4.0 	 2.392052372502783
140 	 185 	 3.0 	 3.198334256321078
140 	 193 	 2.0 	 1.8579708707305123
140 	 208 	 4.0 	 3.98548101794247
140 	 210 	 3.0 	 2.8228711724726683
140 	 246 	 3.0 	 3.124348832388528
140 	 257 	 3.0 	 3.152310730591303
140 	 260 	 4.0 	 2.3834329169151407
140 	 261 	 4.0 	 4.155833566802281
140 	 266 	 3.0 	 3.1094170378206063
140 	 277 	 3.0 	 3.037201388758719
140 	 288 	 2.0 	 1.2568130345893203
140 	 292 	 3.0 	 2.1383612287266858
140 	 293 	 5.0 	 4.677157125630387
140 	 296 	 4.0 	 4.0

140 	 3755 	 4.0 	 3.942889270949384
140 	 3798 	 3.0 	 3.160492795797377
140 	 3827 	 3.0 	 2.959702059848312
140 	 3831 	 3.5 	 3.4312516124379533
140 	 3836 	 4.0 	 3.8276572402424396
140 	 3861 	 2.0 	 2.1935975122470603
140 	 3873 	 4.0 	 3.86134833448883
140 	 3906 	 3.0 	 3.3800133297590857
140 	 3916 	 5.0 	 4.847349804603389
140 	 3948 	 2.5 	 2.849573804618681
140 	 3967 	 3.5 	 3.5849550451402665
140 	 3969 	 3.0 	 3.201210958255941
140 	 3978 	 4.0 	 3.7235718533680693
140 	 3980 	 3.0 	 3.104595910391404
140 	 3983 	 3.0 	 3.1137204981249256
140 	 3985 	 3.5 	 3.383508670681851
140 	 3998 	 4.0 	 4.078311653286587
140 	 3999 	 2.0 	 1.980710322403358
140 	 4011 	 3.0 	 3.378706545179671
140 	 4014 	 4.0 	 3.909862303256043
140 	 4018 	 4.0 	 3.7129221447441676
140 	 4019 	 3.0 	 3.035469240504194
140 	 4022 	 4.0 	 3.833191378331086
140 	 4025 	 4.0 	 3.7884252455978045
140 	 4027 	 2.0 	 2.315954031939248
140 	 4031 	 3.0 	 3.138937956255083
140 	 4033 	 4.0 	 3.793768041